### Let' s create our dataset

In [1]:
# Create a function to setup the directories we'll be storing our images
def makedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass

In [2]:
import cv2
import os

cap = cv2.VideoCapture(0)

i=0
image_count = 0
    
while i < 7:

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    #define region of interest
    roi = frame[100:400, 320:620]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)

    cv2.imshow('roi sacled and gray', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)
    
    if i == 0:
        image_count = 0
        cv2.putText(copy, "Hit Enter to Record when ready", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    if i == 1:
        image_count+=1
        cv2.putText(copy, "Recording no mask - Train", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_one = './facemask/train/0/'
        makedir(gesture_one)
        cv2.imwrite(gesture_one + str(image_count) + ".jpg", roi)
    if i == 2:
        image_count+=1
        cv2.putText(copy, "Recording no mask - Test", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_one = './facemask/test/0/'
        makedir(gesture_one)
        cv2.imwrite(gesture_one + str(image_count) + ".jpg", roi)
    if i == 3:
        cv2.putText(copy, "Hit Enter to Record when ready to Record with mask", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    if i == 4:
        image_count+=1
        cv2.putText(copy, "Recording with mask - Train", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_two = './facemask/train/1/'
        makedir(gesture_two)
        cv2.imwrite(gesture_two + str(image_count) + ".jpg", roi)
    if i == 5:
        image_count+=1
        cv2.putText(copy, "Recording with mask - Test", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_two = './facemask/test/1/'
        makedir(gesture_two)
        cv2.imwrite(gesture_two + str(image_count) + ".jpg", roi)
    if i == 6:
        cv2.putText(copy, "Hit Enter to Exit", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    cv2.imshow('frame', copy)    

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        image_count = 0
        i+=1

cap.release()
cv2.destroyAllWindows() 

In [3]:
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

num_classes = 2
img_rows, img_cols = 28, 28
batch_size = 32

train_data_dir = './facemask/train'
validation_data_dir = './facemask/test'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')

Found 977 images belonging to 2 classes.
Found 560 images belonging to 2 classes.


In [4]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1) ))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.20))

model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

In [5]:
# We use a very small learning rate 
model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

nb_train_samples = 1206 
nb_validation_samples = 301 
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/10
37/37 [==============================] - 1s 20ms/step - loss: 0.6909 - accuracy: 0.5210 - val_loss: 0.6884 - val_accuracy: 0.5625
Epoch 2/10
37/37 [==============================] - 1s 16ms/step - loss: 0.6913 - accuracy: 0.5406 - val_loss: 0.6635 - val_accuracy: 0.6103
Epoch 3/10
37/37 [==============================] - 1s 15ms/step - loss: 0.6866 - accuracy: 0.5423 - val_loss: 0.6623 - val_accuracy: 0.5590
Epoch 4/10
37/37 [==============================] - 1s 16ms/step - loss: 0.6741 - accuracy: 0.5723 - val_loss: 0.5625 - val_accuracy: 0.6618
Epoch 5/10
37/37 [==============================] - 1s 14ms/step - loss: 0.6395 - accuracy: 0.6459 - val_loss: 0.4438 - val_accuracy: 0.9479
Epoch 6/10
37/37 [==============================] - 1s 17ms/step - loss: 0.5903 - accuracy: 0.6820 - val_loss: 0.3313 - val_accuracy: 0.9007
Epoch 7/10
37/37 [==============================] - 1s 17ms/step - loss: 0.5170 - accuracy: 0.7399 - val_loss: 0.1951 - val_accuracy: 0.9514
Epoch 8/10
37

In [6]:
# Save our Model
model.save("face_mask_cnn.h5")

In [7]:
from tensorflow.keras.models import load_model

classifier = load_model('face_mask_cnn.h5')

In [16]:
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    ##############################
    frame=cv2.flip(frame, 1)

    #define region of interest
    roi = frame[100:400, 320:620]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)
    
    cv2.imshow('roi scaled and gray', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)
    
    roi = roi.reshape(1,28,28,1) 
    roi = roi/255
    result = classifier.predict_classes(roi, 1, verbose = 0)[0]
    resultToShow = "No Mask"
    if result[0] == 1:
        resultToShow = "Mask"
    cv2.putText(copy, resultToShow, (300 , 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
    cv2.imshow('frame', copy)    
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 

TypeError: 'NoneType' object is not subscriptable